In [11]:
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import ConversableAgent, AssistantAgent
import newsapi
import os

import datetime
today = datetime.datetime.now().date()

In [2]:
coder_config = {"model": "gpt-4-turbo",
                "max_retries": 10,
                "timeout": 61}
llm_config = {"model": "gpt-4o",
              "max_retries": 10,
              "timeout": 61}

TASK = f"""Write a financial report based on the recent news and stock performance of AVGO for the past four weeks.
The report should include summary of recent events relevant to the company, overall performance of the stock and conclude with recommendation.
Feel free to use newsapi with API key stored in environment variable as NEWS_API_KEY and use up to 5 articles. \
Use yfinance for stock data. Note that there is no Adj Close column for yfinance.\
Write a report using the news and stock data acquired, and then give the report to Engineer to save it as a markdown file. \
Today's date is {today}."""


In [3]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="""Provide a task to be completed. Send instructions to writer to refine the report.""",
    description="A user and administrator. Provides a task to be completed and answer questions if needed.",
    human_input_mode="ALWAYS",
    llm_config=llm_config,    
    code_execution_config=False,
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="""Your role is to effectively plan and coordinate tasks to ensure successful completion. Given any task:
Identify Requirements: Determine what specific information is required to complete the task. Ensure that all suggested information can be retrieved through Python code execution.
Define Steps and Milestones: Break down the task into clear, actionable steps and milestones. Provide detailed instructions for each step to achieve the final goal.
Monitor Progress: After each step is completed by others, review the progress and provide updated guidance on the remaining steps.
Handle Failures: If any step fails, analyze the issue and propose a workaround to keep the task moving forward.
Your objective is to ensure that tasks are completed efficiently by providing well-structured plans, monitoring execution, and troubleshooting issues in real time.
However, you do not write a code and any code wiritng will be done by Engineer.
""",
    description="""Planner. Given a task, identify requirements and devise a plan to complete the task.
    After each step is done by others, check the progress and instruct the remaining steps.""",
    human_input_mode="NEVER",
    llm_config=llm_config,
)

engineer = autogen.ConversableAgent(
    name="Engineer",
    system_message="""Your role is to write code to acquire data based on the detailed plans provided by the Planner.
Code Implementation: Develop well-structured, efficient Python script to achieve the objectives in accordance with the plan. Make sure to write a complete script with every necessary libraries imported. Print the acquired data. Save the script for record.
Error Handling: If the code encounters an error, your primary responsibility is to identify the issue and fix it. This includes debugging, optimizing, and refining the code as needed. Rewrite the entire portion of the script that returned error so that executor can run it end to end.
Intermediary Outputs: If necessary to diagnose the root cause of an error, generate and output intermediary results for analysis. Use these outputs to better understand the problem and guide your debugging process.
Your goal is to ensure that all code runs successfully, handling any errors encountered along the way and iteratively improving the solution until it meets the desired outcome.
""",
    description="""A software engineer that writes code based on the plan provided by the planner.
    When a code returns, Engineer will handle it and fix the error. 
    Should you require an intermediary output to determine the root cause of an error, feel free to output a part for analysis.""",    
    human_input_mode="NEVER",
    llm_config=coder_config,
)

executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Your role is to execute the code written by the Engineer and report the result.",
    description="Executor that runs the code written by the Engineer and report the result to Planner and Admin.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "codes",
        "use_docker": False,
    }
)

writer = autogen.ConversableAgent(
    name="Writer",
    system_message="""You are a writer and financial analyst responsible for creating comprehensive financial reports.
Report Writing: Draft detailed financial reports in Markdown format. Ensure that your reports are structured, clear, and professional. Place the content within pseudo ```md code blocks to indicate Markdown formatting.
Iterative Refinement: Collaborate with the Admin by incorporating their feedback. Refine your report based on the provided suggestions to ensure accuracy, clarity, and completeness.
Your objective is to deliver high-quality financial reports through an iterative process, continually improving the content based on feedback until it meets the required standards.""",
    description="Writer. Write a financial report based on the news and code execution results, and take feedback from the admin to refine the report.",
    human_input_mode="NEVER",
    llm_config=llm_config,    
)


In [4]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=50,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


In [5]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=TASK
)

Admin (to chat_manager):

Write a financial report based on the recent news and stock performance of AVGO for the past four weeks.
The report should include summary of recent events relevant to the company, overall performance of the stock and conclude with recommendation.
Feel free to use newsapi with API key stored in environment variable as NEWS_API_KEY and use up to 5 articles. Use yfinance for stock data. Note that there is no Adj Close column for yfinance.Write a report using the news and stock data acquired, and then give the report to Engineer to save it as a markdown file. Today's date is 2025-01-08.

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

To effectively complete the task of writing a financial report on AVGO, we will proceed through the following structured steps:

### Step 1: Gather News Articles
**Objective:** Use NewsAPI to gather relevant news about AVGO from the past four weeks.

Replying as Admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Proceed to save it


Admin (to chat_manager):

Proceed to save it

--------------------------------------------------------------------------------

Next speaker: Engineer

Engineer (to chat_manager):

Engineer:

### Step 5: Save the Report

Now, as per the instructions, we will save the financial report as a markdown file. I will implement and execute the Python code block that accomplishes this task.

```python
report_content = """
# Financial Report on Broadcom Inc. (AVGO)

**Date:** January 8, 2025

## Introduction
Broadcom Inc. (AVGO) is a prominent player in the semiconductor industry known for its innovative technology solutions. The past four weeks have seen significant developments impacting its market performance.

## Recent Events
Recent news highlights Broadcom's strong position in the AI sector, with reports of achieving a $1 trillion market cap and considerable revenue growth from AI applications. Market analysts emphasize optimism for the semiconductor sector, and Broadcom's favorable positi

Replying as Admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit
